In [90]:
import itertools
import functools
from aoc2024.emulator import Emulator

In [ ]:
emu = Emulator()
emu.load_rom()
emu.run()

# copy the ints
target_vector = [x for x in emu.output]
print("\nThe program output is:", ','.join(map(str, target_vector)))

### Part 2

At this point, I think the brute force approach isn't going to work quick enough.
I started looking for patterns in the output, and I definitely see a repetition when
I continuously bit shift 1 to the left. I suspect that I need at least 45 to 47 bits
in this number in order to produce the same length as the expected output.
This leads me to think that I can limit the search space to just the bits required
in a 45 to 47 bit number. Unfortunately, this means 2^44 combinations to check in
a 45-bit word where there's a 1 in the most significant bit. That is 17,592,186,044,416.
My multiprocess approach is running a million combinations every 27 seconds, so that
means I need to wait 474,989,023 seconds (15.06 years) until I find the solution, and that
is if my assumption is correct.

I decided to stop after 1161000000 combinations.

I'm looking through the emulator instructions, and most of it involves XOR or modulo.
The only time it outputs is when the `out` instruction gets called, and it only takes
the last 3 bits off whatever the result of the combo operator is.

So, if I can figure out how `out` gets called, then maybe I can identify the bits and shift
them to build the number rather than testing all combinations.

In [ ]:

emu = Emulator()
emu.load_rom()
for i in range(63):
    emu.reset()
    number = 1 << i
    for j in range(i):
        number += 1 << j
    emu.rax = number
    emu.run()
    print(i, number, len(emu.output), emu.output)


In [ ]:
target_vector = [ 2,4,1,7,7,5,1,7,0,3,4,1,5,5,3,0]
print("the length of the program is", len(target_vector))

In [ ]:
bit_lengths = [45, 46, 47]


In [ ]:
list(itertools.product([0, 1], repeat=3))

In [ ]:
reversed_target = list(reversed(target_vector))

emu = Emulator()
emu.load_rom()

raxes = []
for target in reversed_target:
    i = 0
    while len(raxes) != len(reversed_target):
        emu.reset()
        emu.rax = i
        emu.run()
        if emu.output and emu.output[0] == target:
            raxes.append(i)
        i += 1
print(raxes)


In [84]:
raxes = []
target = 0

possible=[]
for rax in range(8):
    emu.reset()
    emu.rax = rax
    emu.run()
    print(emu.output)
    if emu.output and emu.output[0] == target:
        possible.append(rax)
if possible:
    raxes.append(possible)

print(raxes)

[0]
[1]
[2]
[3]
[4]
[4]
[5]
[0]
[[0, 7]]


In [85]:
target = 3

possible=[]
for past in raxes[0]:
    print("Try:",past)
    shifted = past << 3
    for rax in range(shifted, shifted+8):
        emu.reset()
        emu.rax = rax
        emu.run()
        print(rax, emu.output)
        if emu.output and emu.output[0] == target:
            possible.append(rax)

if possible:
    raxes.append(possible)
print(raxes)


Try: 0
0 [0]
1 [1]
2 [2]
3 [3]
4 [4]
5 [4]
6 [5]
7 [0]
Try: 7
56 [0, 0]
57 [1, 0]
58 [3, 0]
59 [0, 0]
60 [3, 0]
61 [2, 0]
62 [1, 0]
63 [0, 0]
[[0, 7], [3, 58, 60]]


In [88]:
emu.reset()
emu.rax = 60
emu.run()
emu.output

[3, 0]

In [91]:
reversed_target = list(reversed(target_vector))
print("I need to find these targets", reversed_target)

emu = Emulator()
emu.load_rom()

raxes = [[0]]
for target in reversed_target:
    possible = []

    for past in itertools.chain(*raxes):
        #print("Try:",past)
        shifted = past << 3
        for rax in range(shifted, shifted+8):
            emu.reset()
            emu.rax = rax
            emu.run()
            #print(rax, emu.output)
            if emu.output and emu.output[0] == target:
                possible.append(rax)
    if possible:
        raxes.append(possible)
print(raxes)



I need to find these targets [0, 3, 5, 5, 1, 4, 3, 0, 7, 1, 5, 7, 7, 1, 4, 2]
[[0], [0, 7], [3, 3, 58, 60], [6, 6, 470, 482, 483, 486], [6, 6, 470, 482, 483, 486, 54, 54, 3760, 3766, 3857, 3862, 3865, 3889, 3894], [1, 1, 57, 62, 25, 30, 25, 30, 49, 49, 3870, 49, 49, 3870, 30856, 30862, 30896, 30920, 30926, 31118], [4, 5, 4, 5, 466, 485, 30084, 30085, 30922, 458, 499, 202, 243, 202, 243, 30963, 30963, 246852, 246853, 246899, 247172, 247173, 247360, 247364, 247365, 247408, 247411, 248947], [3, 3, 58, 60, 464, 480, 50, 50, 3761, 3890, 50, 50, 3761, 3890, 432, 432, 30080, 30083, 30128, 30857, 30859, 30897, 31112, 31115, 31152, 11, 11, 456, 496, 392, 395, 392, 395, 392, 395, 392, 395, 247168, 247171, 34, 42, 34, 42, 3729, 3882, 240674, 240682, 1974818, 1974826, 1977378, 1977386, 1978883, 1978914, 1978922, 1979267], [0, 7, 0, 7, 56, 59, 63, 24, 31, 24, 31, 469, 471, 484, 487, 48, 51, 53, 55, 48, 51, 53, 55, 3763, 3765, 3767, 3861, 3863, 3871, 3891, 3893, 3895, 48, 51, 53, 55, 48, 51, 53, 55,

In [92]:
for rax in itertools.chain(*raxes):
    emu.reset()
    emu.rax = rax
    emu.run()
    if emu.output == target_vector:
        print("The solution is", rax)
        break

THe solution is 265601188299675
